In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gdsc-aiml-member-recruitments/train.csv
/kaggle/input/gdsc-aiml-member-recruitments/test.csv
/kaggle/input/gdsc-aiml-member-recruitments/sample_solution.csv


In [21]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

In [22]:
file_path="/kaggle/input/gdsc-aiml-member-recruitments/train.csv"
weather_data=pd.read_csv(file_path)
weather_data.head()

,ID,YEAR,T2M_RANGE,T2M_MAX,T2M_MIN,RH2M,PS,WS10M,QV2M,T2M,VACATION_RATE
0,0,1982.0,-241.552430,29.798545,13.691128,71.652934,89.725069,3.658959,8.474319,21.723403,64.184741
1,1,1982.0,-241.819997,30.902708,15.298046,69.754933,89.865668,3.529113,8.988374,22.911522,42.649137
2,2,1982.0,-241.883202,30.296561,14.660419,66.661416,89.846273,4.217007,8.474923,22.994887,52.819141
3,3,1982.0,-242.007629,30.803224,15.246491,67.621173,90.029795,4.484080,9.005508,23.594655,39.445489
4,4,1982.0,-240.659332,32.500906,15.303059,62.097329,89.825707,3.057211,8.307515,24.536776,46.444289


In [23]:
weather_data.columns

Index(['ID', 'YEAR', 'T2M_RANGE', 'T2M_MAX', 'T2M_MIN', 'RH2M', 'PS', 'WS10M',
       'QV2M', 'T2M', 'VACATION_RATE'],
      dtype='object')

In [24]:
features=["YEAR","T2M_RANGE","T2M_MAX","T2M_MIN","RH2M","PS","WS10M"]
X=weather_data[features]

In [25]:
targets=["T2M","QV2M","VACATION_RATE"]
y=weather_data[targets]

In [41]:
param_grid = {
    'learning_rate': [0.01, 0.1, 0.1],
    'max_depth': [5, 4, 4],
    'n_estimators': [700, 600, 500],
    'subsample': [0.5, 0.5, 0.5],
    'colsample_bytree' : [0.1, 0.1, 0.1]
}


In [ ]:
xgb_reg = xgb.XGBRegressor(random_state=42)
grid_search = GridSearchCV(estimator=xgb_reg, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X,y)

In [36]:
best_params = grid_search.best_params_
best_xgb_model = grid_search.best_estimator_

In [37]:
test_file_path='/kaggle/input/gdsc-aiml-member-recruitments/test.csv'
weather_data_test=pd.read_csv(test_file_path)
weather_data_test.columns

Index(['ID', 'YEAR', 'T2M_RANGE', 'T2M_MAX', 'T2M_MIN', 'RH2M', 'PS', 'WS10M'], dtype='object')

In [38]:
test_features=["YEAR","T2M_RANGE","T2M_MAX","T2M_MIN","RH2M","PS","WS10M"]
X_test=weather_data_test[test_features]

In [39]:
y_pred = best_xgb_model.predict(X_test)
print(y_pred)

[[25.133598 11.997922 29.223005]
 [25.175346 12.406047 27.937592]
 [24.247786 12.801599 21.1562  ]
 ...
 [20.791826 11.614451 46.558624]
 [20.118002 11.516527 76.148186]
 [21.714975 12.291968 55.048325]]


In [40]:
vacation_rate = [row[-1] for row in y_pred]
#CREATING THE LIST OF IDS
ids = [1440 + i for i in range(len(vacation_rate))]

#MAPPING EACH ID WITH EACH VALUE OF VACATION_RATE
data = list(zip(ids, vacation_rate))

#CREATING A NEW CSV FILE CONTAINING ID AND VACATION_RATE ONLY
csv_file_path = "/kaggle/working/xgfile.csv"
with open(csv_file_path, mode="w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["ID", "VACATION_RATE"])  # Write header
    csv_writer.writerows(data)  # Write data rows
print("CSV file has been created.")

CSV file has been created.
